In [16]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import random
import warnings
warnings.simplefilter('ignore')

In [14]:
data_grouping=pd.read_csv("data_with_grouping_operations_IS.csv")
df.head(10)

,city,price,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,lat,long,state,predicted_state,state_comb
0,"abilene, TX",9000,2009,chevrolet,suburban lt2,good,8,gas,217743.0,clean,automatic,rwd,SUV,33.130100,-100.234400,TX,TX,TX
1,"abilene, TX",31999,2012,ram,2500,NaN,0,diesel,NaN,clean,automatic,NaN,NaN,30.648368,-97.862949,TX,TX,TX
2,"abilene, TX",16990,2003,ram,3500,NaN,0,diesel,NaN,clean,manual,NaN,NaN,30.648516,-97.862434,TX,TX,TX
3,"abilene, TX",6000,2002,gmc,sierra 1500,good,8,gas,195000.0,clean,automatic,4wd,pickup,32.444400,-99.992400,TX,TX,TX
4,"abilene, TX",37000,2012,chevrolet,3500,excellent,8,diesel,178000.0,clean,automatic,4wd,pickup,32.781689,-98.942248,TX,TX,TX
5,"abilene, TX",3700,2003,NaN,F150,fair,8,gas,269000.0,clean,automatic,4wd,pickup,32.579641,-99.663505,TX,TX,TX
6,"abilene, TX",19950,2013,ford,f-250,NaN,8,gas,116792.0,clean,automatic,4wd,pickup,32.736030,-97.133644,TX,OK,TX
7,"abilene, TX",19999,2006,ram,2500,NaN,0,diesel,NaN,clean,manual,NaN,NaN,30.647777,-97.862434,TX,TX,TX
8,"abilene, TX",33950,2015,ford,f-350,NaN,8,diesel,77350.0,clean,automatic,4wd,pickup,32.754000,-97.134800,TX,OK,TX
9,"abilene, TX",25950,2015,ford,f-350,NaN,8,gas,121030.0,clean,automatic,4wd,truck,32.736030,-97.133644,TX,OK,TX


In [17]:
train_proportion = 0.8
features_colname=['year','odometer','income_rank','median_income',
       'cylinders_onehot_1', 'cylinders_onehot_2', 'cylinders_onehot_3',
       'cylinders_onehot_4', 'cylinders_onehot_5', 'cylinders_onehot_6',
       'cylinders_onehot_7', 'cylinders_onehot_8', 'type_onehot_1',
       'type_onehot_2', 'type_onehot_3', 'type_onehot_4', 'type_onehot_5',
       'type_onehot_6', 'type_onehot_7', 'type_onehot_8', 'type_onehot_9',
       'type_onehot_10', 'type_onehot_11', 'type_onehot_12', 'type_onehot_13',
       'type_onehot_14','drive_onehot_1', 'drive_onehot_2', 'drive_onehot_3',
       'drive_onehot_4', 'title_status_mean_price','manufacturer_mean_price','manufacturer_make_mean_price','transmission_encoded', 'fuel_encoded']
data_clean=data_grouping[features_colname+["price"]]

# #fill nans in features
# features=features.fillna(features.mean())

#  drop nan
data_clean=data_clean.dropna(axis = 0, how ='any')
print(len(data_clean))
features=data_clean[features_colname]

# scaling
features['odometer']=(features['odometer']-features['odometer'].mean())/features['odometer'].std()
features['median_income']=(features['median_income']-features['median_income'].mean())/features['median_income'].std()
features['manufacturer_mean_price']=(features['manufacturer_mean_price']-features['manufacturer_mean_price'].mean())/features['manufacturer_mean_price'].std()
features['manufacturer_make_mean_price']=(features['manufacturer_make_mean_price']-features['manufacturer_make_mean_price'].mean())/features['manufacturer_make_mean_price'].std()
features['income_rank']=features['income_rank']/features['income_rank'].max()
target=data_clean["price"]

index_list=[i for i in range(len(data_clean))]

random.shuffle(index_list)
train_list=index_list[:round(len(data_clean)*train_proportion)]
test_list=index_list[round(len(data_clean)*train_proportion):]

# the following variable records the features of examples in the training set
train_x=features.iloc[train_list]
# the following variable records the features of examples in the test set
test_x=features.iloc[test_list]
# the following variable records the labels of examples in the training set
train_y=target.iloc[train_list]
# the following variable records the labels of examples in the test set
test_y=target.iloc[test_list]

326105


In [19]:
def MAE_relative(y,pred):# assume input as arrays
    return abs(1-pred/y).mean()

# Lasso

In [18]:
lasso = linear_model.Lasso(alpha=0.1)
lasso.fit(train_x,train_y)
print(lasso.coef_)
print(lasso.intercept_)

[ 5.74904997e+02 -2.76866172e+03 -5.01196128e+02 -2.16058525e+02
  4.96948036e+02 -2.15393813e+03 -1.39582772e+03 -8.90950223e+02
 -5.49116406e+01  6.40185087e+02  9.63570664e+02  8.23015515e+03
 -6.56310678e+02  1.97345331e+03  2.21778027e+03  9.34390972e+02
 -1.04996979e+03  2.30149833e+02 -1.34338953e+02  1.12188288e+03
 -5.85995368e+02  1.16943917e+03 -7.23433499e+02  1.99502879e+02
 -0.00000000e+00 -9.80502680e+02  2.22116131e+03 -4.69490371e+02
 -0.00000000e+00  2.36990457e+02  5.68349584e-01  2.81814135e+02
  5.42679580e+03 -3.67124049e+02 -9.84772657e+02]
-1147495.4718909445


In [20]:
train_pred = lasso.predict(train_x)
test_pred = lasso.predict(test_x)

train_MAE_relative = MAE_relative(train_y,train_pred)
test_MAE_relative = MAE_relative(test_y,test_pred)

print("Train relative error:",train_MAE_relative)
print("Test relative error:",test_MAE_relative)

Train relative error: 0.4992265726643765
Test relative error: 0.49265964132920126


# Huber

In [21]:
huber = linear_model.HuberRegressor().fit(train_x, train_y)
print(huber.coef_)
print(huber.intercept_)

[ 3.53876889e+00 -4.83739008e+03  2.26611956e+01 -1.57846111e+02
  6.77691212e+02 -3.28697212e+00 -5.40506062e+02 -4.31171499e+01
 -2.56579751e+02  1.45168034e+02 -2.19582731e+00 -3.02407193e-01
 -1.02911452e+01  1.18337186e-01 -5.44766015e+01 -1.40190080e+01
 -1.16473629e+02 -2.65490954e+01 -2.88414091e+02 -5.96416057e+00
  1.02801510e+02  5.92642380e+02 -4.96546506e+02  3.49621862e+02
 -9.01634294e-01 -5.46771419e+01  1.12660166e+03 -5.91305959e+02
 -2.90715297e+02 -2.67709324e+02  5.49057292e-01  5.50239496e+02
  5.46782474e+03  8.86170089e+01 -5.40772943e+02]
-23.128963153193453


In [22]:
train_pred = huber.predict(train_x)
test_pred = huber.predict(test_x)

train_MAE_relative = MAE_relative(train_y,train_pred)
test_MAE_relative = MAE_relative(test_y,test_pred)

print("Train relative error:",train_MAE_relative)
print("Test relative error:",test_MAE_relative)

Train relative error: 0.5317582689239112
Test relative error: 0.5231762633563081
